In [6]:
## Can also get the price chart by pokemon number, or url for past prices. Look intro trading strategies; also, work on real-time offering feed vs history. 

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import re

num_pages = 5
base_url = 'https://www.tcgplayer.com/search/pokemon/product?productLineName=pokemon&view=grid&page={}'

options = Options()
options.add_argument("--headless")
chrome_prefs = {"profile.default_content_setting_values": {"images": 2, "stylesheets": 2}}
options.experimental_options["prefs"] = chrome_prefs

def scrape_page(page):
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    url = base_url.format(page)
    driver.get(url)
    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card__product")))
    except:
        driver.quit()
        return []

    cards = driver.find_elements(By.CLASS_NAME, "product-card__product")
    page_data = []
    for card in cards:
        try:
            img_elem = card.find_element(By.TAG_NAME, "img")
            name = img_elem.get_attribute("alt").strip()
            src = img_elem.get_attribute("src")
            match = re.search(r"(https://tcgplayer-cdn\.tcgplayer\.com/product/\d{6})", src)
            link = match.group(1) if match else src
        except Exception:
            name = None
            link = None

        try:
            set_elem = card.find_element(By.CLASS_NAME, "product-card__set-name__variant")
            set_name = set_elem.text.strip()
        except Exception:
            set_name = None

        try:
            price_elem = card.find_element(By.CLASS_NAME, "product-card__market-price--value")
            mktprice = price_elem.text.strip()
        except Exception:
            mktprice = None

        page_data.append({
            "name": name,
            "link": link,
            "set": set_name,
            "mktprice": mktprice
        })

    driver.quit()
    return page_data

all_data = []
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(scrape_page, page) for page in range(1, num_pages + 1)]
    for future in as_completed(futures):
        all_data.extend(future.result())

pricedf = pd.DataFrame(all_data)
print(pricedf)


                                   name  \
0               Boss's Orders - 172/193   
1                    Vivillon - 007/191   
2                              N's Plan   
3                                Mewtwo   
4                               Groudon   
5                                Dreepy   
6                             N's PP Up   
7                 Venusaur ex - 003/165   
8                              Drakloak   
9                         Bravery Charm   
10                        Jamming Tower   
11                        Bouffalant ex   
12  Mega Evolution Enhanced Booster Box   
13                          Meloetta ex   
14                    Wugtrio - 224/091   
15                          Reshiram ex   
16               Blastoise ex - 009/165   
17                            Keldeo ex   
18                        Whimsicott ex   
19                              Umbreon   
20                  Bulbasaur - 001/165   
21            Paradox Rift Booster Pack   
22         

In [8]:
pricedf

,name,link,set,mktprice
0,Boss's Orders - 172/193,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV02: Paldea Evolved,$1.54
1,Vivillon - 007/191,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV08: Surging Sparks,$0.08
2,N's Plan,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV: Black Bolt,$0.12
3,Mewtwo,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV: Scarlet & Violet 151,$0.55
4,Groudon,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",Celebrations,$0.18
5,Dreepy,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV: Prismatic Evolutions,$0.10
6,N's PP Up,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV09: Journey Together,$0.13
7,Venusaur ex - 003/165,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV: Scarlet & Violet 151,$0.74
8,Drakloak,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV: Prismatic Evolutions,$0.11
9,Bravery Charm,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",SV02: Paldea Evolved,$1.12
